In [1]:
from Bio.PDB import *

from scipy.spatial.transform import Rotation as R

from utils.carb_utils import *


#-alternate_3_letter_codes pdb_sugar
#-out:level 100

#-beta
#-auto_detect_glycan_connections
#-alternate_3_letter_codes pdb_sugar



import os
import numpy as np
import pandas as pd
import copy

input_dir = "./"
os.chdir(input_dir)

In [ ]:
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *

from pyrosetta.rosetta.protocols.carbohydrates import *
from pyrosetta.rosetta.core.select.residue_selector import *
from pyrosetta.rosetta.core.simple_metrics.metrics import *
from pyrosetta.rosetta.core.simple_metrics.composite_metrics import *
from pyrosetta.rosetta.core.simple_metrics.per_residue_metrics import *

options = """
-ignore_unrecognized_res
-include_sugars
-auto_detect_glycan_connections
-maintain_links 
-alternate_3_letter_codes pdb_sugar
-write_glycan_pdb_codes
-ignore_zero_occupancy false 
-load_PDB_components false
-no_fconfig

-out:level 100
"""

init(" ".join(options.split('\n')))

In [ ]:
sc = get_score_function()
fr = pyrosetta.rosetta.protocols.relax.FastRelax()
fr.set_scorefxn(sc)
fr.max_iter(100)

In [2]:
#Get list of proteins
pdb = [];

df = pd.read_csv('./carbbinders_pdblist.txt',header=None).values

print(len(df),df)

6559 [['3old']
 ['3ole']
 ['1pig']
 ...
 ['5DFM']
 ['4A34']
 ['5HQJ']]


In [3]:
ls = os.listdir('./pdbs')
print(len(ls))

6251


In [4]:
gly = '''N-acetyl-beta-D-glucosamine (NAG), beta-D-glucopyranose (BGC),
alpha-D-glucopyranose (GLC), alpha-D-mannopyranose (MAN),
beta-D-galactopyranose (GAL), alpha-L-fucopyranose (FUC),
beta-D-mannopyranose (BMA), N-acetyl-alpha-D-mannosamine (BM3), nonyl
beta-D-glucopyranoside (BNG), beta-D-xylopyranose (XYP),
uridine-diphosphate-n-acetylglucosamine (UD1), N-acetyl-alpha-D-galactosamine
(A2G), undecyl-maltoside (UMQ), N-acetyl-beta-D-galactosamine (NGA),
galactose-uridine-5-diphosphate (GDU), 6-O-phosphono-beta-D-glucopyranose
(BG6), N-acetyl-beta-neuraminic acid (SLB), N-acetyl-alpha-neuraminic acid (SIA), (NEU),
fructose -6-phosphate (F6R), alpha-D-galactopyranose (X6X),
1,6-di-O-phosphono-D-fructose (P6F), 2-amino-2-deoxy-alpha-D-glucopyranose
(PA1), 2-amino-2-deoxy-beta-D-galactopyranose (1GN), alpha-L-rhamnopyranose
(RAM), alpha-D-Abequopyranose (ABE), alpha-D-glucopyranuronic acid (GCU),
2-acetamido-2-deoxy-4-O-sulfo-beta-D-galactopyranose (ASG),
alpha-L-gulopyranuronic acid (LGU), alpha-L-arabinofuranose (AHR),
beta-D-galactofuranuronic acid (GTK), beta-D-glucopyranuronic acid (BDP),
beta-L-fructofuranose (LFR) and cyclodextrins'''

carb = [];
gly = gly.split('(')
#print(gly)
for jj in range(len(gly)):
    if ')' in gly[jj]:
        carb.append(gly[jj].split(')')[0])
print(carb)

['NAG', 'BGC', 'GLC', 'MAN', 'GAL', 'FUC', 'BMA', 'BM3', 'BNG', 'XYP', 'UD1', 'A2G', 'UMQ', 'NGA', 'GDU', 'BG6', 'SLB', 'SIA', 'NEU', 'F6R', 'X6X', 'P6F', 'PA1', '1GN', 'RAM', 'ABE', 'GCU', 'ASG', 'LGU', 'AHR', 'GTK', 'BDP', 'LFR']


In [29]:
aa_inv = {
    "H": "HIS",
    "K": "LYS",
    "R": "ARG",
    "D": "ASP",
    "E": "GLU",
    "S": "SER",
    "T": "THR",
    "N": "ASN",
    "Q": "GLN",
    "A": "ALA",
    "V": "VAL",
    "L": "LEU",
    "I": "ILE",
    "M": "MET",
    "F": "PHE",
    "Y": "TYR",
    "W": "TRP",
    "P": "PRO",
    "G": "GLY",
    "C": "CYS",
    "X": "MSE",
    "Z": "CYD"
}
aa_inv.keys()
prot_aa = aa_inv.values()

In [5]:
parser=PDBParser()

structure=parser.get_structure("prot", 'pdbs/'+ ls[0])

/Users/scanner1/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 16199.
  warnings.warn(
/Users/scanner1/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 16283.
  warnings.warn(
/Users/scanner1/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 16311.
  warnings.warn(
/Users/scanner1/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 16395.
  warnings.warn(
/Users/scanner1/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 16423.
  warnings.warn(
/Users/scanner1/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstru

In [22]:
class GlycanSelect(Select):
    def accept_residue(self, residue):
        r = residue
        #help(r)
        #if residue.get_resname()=='GLY':
        if residue.get_resname() in carb:
            return 1
        else:
            return 0
        
io=PDBIO()
io.set_structure(structure)
io.save('gly_only.pdb', GlycanSelect())

In [30]:
class ProteinSelect(Select):
    def accept_residue(self, residue):
        r = residue
        #help(r)
        #if residue.get_resname()=='GLY':
        if residue.get_resname() in prot_aa:
            return 1
        else:
            return 0
        
io=PDBIO()
io.set_structure(structure)
io.save('prot_only.pdb', ProteinSelect())

In [14]:
penis.get_resname()

'SIA'

In [21]:
kk.get_name()
kk.get_coord()
print( kk.get_charge() )

None


In [16]:
#Determine which ones are glycosylated and which are non-covalent
penis = []
kk = []

coor = []

models = structure.get_models()
models = list(models)
for m in range(len(models)):
    chains = list(models[m].get_chains())
    for c in range(len(chains)):
        residues = list(chains[c].get_residues())
        for r in range(len(residues)):
            res = residues[r].get_resname()
            if res in carb or res in prot_aa:
                #check if its a binding residue
                #temp = 1.00
                #penis = residues[r]
                #its a predicted residue -> BFactor = 99.99
                print(str(residues[r].id[1]).strip(),str(chains[c].id).strip(), str(residues[r].get_resname() ))

                atoms = list(residues[r].get_atoms())
                for a in range(len(atoms)):
                    #print(atoms[a].)

                    at = atoms[a]
                    coor.append( at.get_coord() )
            
            #my_res = str(residues[r].id[1]).strip() + "." + str(chains[c].id).strip()
            #print(my_res)
#Determine interacting chains (e.g. remove redundant chains)


#Save by glycan chain

#save by pdb-glycan interacting chain
#svae FASTA of pdb-glycan buddy
#save complete FASTA

1 A GLN
2 A ASP
3 A LEU
4 A PRO
5 A GLY
6 A ASN
7 A ASP
8 A ASN
9 A SER
10 A THR
11 A ALA
12 A THR
13 A LEU
14 A CYS
15 A LEU
16 A GLY
17 A HIS
18 A HIS
19 A ALA
20 A VAL
21 A PRO
22 A ASN
23 A GLY
24 A THR
25 A LEU
26 A VAL
27 A LYS
28 A THR
29 A ILE
30 A THR
31 A ASP
32 A ASP
33 A GLN
34 A ILE
35 A GLU
36 A VAL
37 A THR
38 A ASN
39 A ALA
40 A THR
41 A GLU
42 A LEU
43 A VAL
44 A GLN
45 A SER
46 A SER
47 A SER
48 A THR
49 A GLY
50 A LYS
51 A ILE
52 A CYS
53 A ASN
54 A ASN
55 A PRO
56 A HIS
57 A ARG
58 A ILE
59 A LEU
60 A ASP
61 A GLY
62 A ILE
63 A ASP
64 A CYS
65 A THR
66 A LEU
67 A ILE
68 A ASP
69 A ALA
70 A LEU
71 A LEU
72 A GLY
73 A ASP
74 A PRO
75 A HIS
76 A CYS
77 A ASP
78 A VAL
79 A PHE
80 A GLN
81 A ASN
82 A GLU
83 A THR
84 A TRP
85 A ASP
86 A LEU
87 A PHE
88 A VAL
89 A GLU
90 A ARG
91 A SER
92 A LYS
93 A ALA
94 A PHE
95 A SER
96 A ASN
97 A CYS
98 A TYR
99 A PRO
100 A TYR
101 A ASP
102 A VAL
103 A PRO
104 A ASP
105 A TYR
106 A ALA
107 A SER
108 A LEU
109 A ARG
110 A SER
111 A LE

In [6]:

        
#Determines if the protein is glycosylated or we have free carbohydrates
def is_glycosylated(pose):
    """
    Gets whether a pose contains glycosylated proteins
    Args:
        pose : pyrosetta pose 
    Returns:
        bool : True / False
    """
    tree_set = pose.glycan_tree_set()
    for start in tree_set.get_start_points():
        parent = tree_set.get_parent(start)
        if parent != 0:
            return True;
    return False
    

In [7]:
os.getcwd()

'/Users/scanner1/Downloads/capdock'

In [8]:
ls[0]

'1HGG.pdb'

In [9]:
pose = pose_from_pdb('./pdbs/' + ls[0])


ERROR: Residue::carbohydrate_info(): This residue is not a carbohydrate!
ERROR:: Exit from: /Volumes/scratch/w/rosetta/commits/rosetta/source/src/core/conformation/Residue.cc line: 419


RuntimeError: 

File: /Volumes/scratch/w/rosetta/commits/rosetta/source/src/core/conformation/Residue.cc:419
[ ERROR ] UtilityExitException
ERROR: Residue::carbohydrate_info(): This residue is not a carbohydrate!



In [10]:
pose = pyrosetta.toolbox.pose_from_rcsb('1w3l',ATOM=1)

core.import_pose.import_pose: File '1W3L.clean.pdb' automatically determined to be of type PDB
core.io.util: Found 0 glycan linkages in the structure based on atomic distances
core.io.util: Automatic glycan connection is activated.
core.io.util: Start reordering residues.
core.io.util: Corrected glycan residue order (internal numbering): []
core.io.util:
core.io.util: Found 0 glycan linkages in the structure based on atomic distances


In [11]:
pose

In [9]:
ls[0]

'1HGG.pdb'

In [12]:
r = pose.residue(305)
r.is_carbohydrate()

False

In [3]:
tree_set = pose.glycan_tree_set()

In [6]:
print( tree_set )

None


In [4]:
s = []
for start in tree_set.get_start_points():
    #dont include glycosylated bois
    parent = tree_set.get_parent(start);
    #print(parent)
    if parent != 0:
        continue;
    print(start, pose.pdb_info().pose2pdb(start), pose.residue_type(start).name3(), pose.residue_type(start).name())
    s.append(start)

AttributeError: 'NoneType' object has no attribute 'get_start_points'

In [21]:
tree1 = tree_set.get_tree(s[0])

In [22]:
tree1.get_residues()

vector1_unsigned_long[1525, 1526, 1527]

In [23]:
#single test

out = ""

tree_set = pose.glycan_tree_set()
s = []
for start in tree_set.get_start_points():
    #skip the glycosylated glycans
    parent = tree_set.get_parent(start);
    if parent != 0:
        continue;
    #print(start, pose.pdb_info().pose2pdb(start), pose.residue_type(start).name3(), pose.residue_type(start).name())
    s.append(start)

prot = pose.clone()
    
ind = 0;
for i in s:
    tree = tree_set.get_tree(i)
    
    
    res = np.array(tree.get_residues())
    carb = pose.clone()
    for j in range(pose.size()-1,0,-1):
        if j not in res:
            carb.delete_residue_slow(j)
    #print(carb)
    
    
    #save the carb
    carb.dump_pdb('pdb_pre/1bag_carb' + str(ind) + ".pdb")
    
    #relax the carb and save rosetta relax
    #fr.apply(carb)
    #carb.dump_pdb('pdb_pre/1bag_carb' + str(ind) + "_rosRel.pdb")
    
    ind += 1;
    

#have a clean protein file
for j in range(prot.size(),0,-1):
    #print(j)
    if prot.residue(j).is_protein():
        continue;
    print(j)
    prot.delete_residue_slow(j)
    

#output the fasta
for ii in range(1,prot.num_chains()+1):
    out += ">1BAG_" + str(ii) + "\n"
    out += prot.chain_sequence(ii) + "\n"
    
prot.dump_pdb('pdb_pre/1bag_prot.pdb')
#print(out)
    

RuntimeError: 

File: /Volumes/MacintoshHD3/benchmark/W.fujii.release/rosetta.Fujii.release/_commits_/main/source/build/PyRosetta/macos/clang-9.0.0/python-3.8/release/source/src/core/conformation/Conformation.hh:514
[ ERROR ] UtilityExitException
ERROR: Error in core::conformation::Conformation::residue(): The sequence position requested was greater than the number of residues in the pose.



In [36]:
r = pose.residue(3)

In [37]:
r.natoms()

19

In [38]:
r.name3()

'LEU'

In [45]:
pose.beta(3)

RuntimeError: 

File: /Volumes/MacintoshHD3/benchmark/W.fujii.release/rosetta.Fujii.release/_commits_/main/source/src/core/pose/Pose.cc:1444
[ ERROR ] UtilityExitException
ERROR: Pose::beta( Size const seqpos ): residue seqpos is not part of a Nucleic Acid!



In [50]:
p = pose.pdb_info()
p.bfactor(4,2)

135.18

In [19]:
for i in range(1,20):
    print(r.atom_name(i))

 N  
 CA 
 C  
 O  
 CB 
 CG 
 CD1
 CD2
 H  
 HA 
1HB 
2HB 
 HG 
1HD1
2HD1
3HD1
1HD2
2HD2
3HD2


In [9]:
def dump_res_pdb(pose,res,file):
    
    
    anum = 1;
    resnum = 1;
    out = ''

    for ii in res:
        
        r = pose.residue(ii)
        resnum = r.pose2

        for a in range(1,r.natoms()+1):
            
            aname = r.atom_name(a)
            if 'H' in aname:
                continue;
            if 'V' in aname:
                continue;
            
            out += "ATOM  "
            out += str(anum).rjust(5) + ' '
            out += aname
            out += ' ' # no alt location indicator
            out += r.name3().ljust(4)
            out += 'A'
            out += str(ii).rjust(4)
            out += '   '
            #print(m.coor[a,:])
            coor = r.xyz(a)
            c = str( round(coor[0],3) )
            while len( c.split('.')[1] ) < 3:
                c += '0'
            out += c.rjust(8)
            c = str( round(coor[1],3) )
            while len( c.split('.')[1] ) < 3:
                c += '0'
            out += c.rjust(8)
            c = str( round(coor[2],3) )
            while len( c.split('.')[1] ) < 3:
                c += '0'
            out += c.rjust(8)


            #out += str( round(m.coor[a,1],3) ).rjust(8)
            #out += str( round(m.coor[a,2],3) ).rjust(8)

            out += '\n'
            anum += 1
        resnum += 1

    
    return out
    

In [10]:
def output_pdbs(name,pose):

    out = ""

    #get the number of trees
    tree_set = pose.glycan_tree_set()
    s = []
    for start in tree_set.get_start_points():
        #skip the glycosylated glycans
        parent = tree_set.get_parent(start);
        if parent != 0:
            continue;
        #print(start, pose.pdb_info().pose2pdb(start), pose.residue_type(start).name3(), pose.residue_type(start).name())
        s.append(start)

    #print(pose.size())
    #prot = pose.clone()

    ind = 0;
    for i in s:
        tree = tree_set.get_tree(i)
        res = np.array(tree.get_residues())
        #carb = pose.clone()
        

        print( dump_res_pdb(pose,res,'') )

        #print(carb)


        #save the carb
        #carb.dump_pdb('pdb_pre/' + name + '_carb' + str(ind) + ".pdb")

        #relax the carb and save rosetta relax
        #fr.apply(carb)
        #carb.dump_pdb('pdb_pre/' + name + '_carb' + str(ind) + "_rosRel.pdb")

        #ind += 1;

    #print(pose.size(),carb.size(),prot.size())
    #have a clean protein file
    for j in range(prot.size(),0,-1):
        #print(j)
        if j == 0:
            break;
        if prot.residue(j).is_protein():
            continue;
        #print(j)
        prot.delete_residue_slow(j)

    #print(pose.size(),carb.size(),prot.size())
    #output the fasta
    for ii in range(1,prot.num_chains()+1):
        out += ">" + name + "_" + str(ii) + "\n"
        out += prot.chain_sequence(ii) + "\n"

    prot.dump_pdb('pdb_pre/' + name + '_prot.pdb')
    #print(out)
    return out
    
output_pdbs('1hgg',pose)

NameError: name 'pose' is not defined

In [ ]:
#output_pdbs('1bag',pyrosetta.toolbox.pose_from_rcsb('1bag'.upper(),ATOM=False))
#output_pdbs('1hgg',pose)

In [18]:
fa = ''
for ii in ls[:10]:

    #print(ii[0])
    #try:
        #pdb = ii[0]
        pdb = ii
        print(pdb)
        #pose = pyrosetta.toolbox.pose_from_rcsb(pdb.upper(),ATOM=False)
        pose = pose_from_pdb('./pdbs/' + pdb)
        fa += output_pdbs(pdb,pose)

        f = open('pdbs/fasta.fa','w+')
        f.write(fa)
        f.close()
    #except:
    #    print("Unable: ",pdb)
    
    #break;

1HGG.pdb



ERROR: Residue::carbohydrate_info(): This residue is not a carbohydrate!
ERROR:: Exit from: /Volumes/scratch/w/rosetta/commits/rosetta/source/src/core/conformation/Residue.cc line: 419


RuntimeError: 

File: /Volumes/scratch/w/rosetta/commits/rosetta/source/src/core/conformation/Residue.cc:419
[ ERROR ] UtilityExitException
ERROR: Residue::carbohydrate_info(): This residue is not a carbohydrate!

